In [1]:
import requests  # Importation du module requests pour faire des requêtes HTTP
import datetime  # Importation du module datetime pour travailler avec des dates et des heures

In [2]:
# URL de base de l'API OpenWeatherMap pour obtenir les données météorologiques
base_url = "http://api.openweathermap.org/data/2.5/weather"

In [3]:
# Votre clé API pour accéder à l'API OpenWeatherMap (remplacez-la par la vôtre)
api_key = 'XXX'

In [4]:
# Nom de la ville pour laquelle on souhaite obtenir les données météorologiques
city_name = 'Paris'

In [5]:
# Paramètres de la requête : nom de la ville, clé API et unités métriques pour les données
params = {
    'q': city_name,
    'appid': api_key,
    'units': 'metric'  # Pour obtenir les données en unités métriques
}

In [6]:
# Envoi de la requête GET à l'API OpenWeatherMap avec les paramètres spécifiés
response = requests.get(base_url, params=params)

In [7]:
# Vérification du statut de la réponse pour s'assurer qu'elle est réussie (code 200)
if response.status_code == 200:
    # Conversion de la réponse JSON en dictionnaire Python
    data = response.json()

    # Extraction des informations principales de la réponse
    main = data['main']
    weather = data['weather'][0]
    localtime = datetime.datetime.utcfromtimestamp(data['dt']) + datetime.timedelta(seconds=data['timezone'])

    current_weather_data = {
        'city': data['name'],              # Nom de la ville
        'temperature': main['temp'],       # Température actuelle
        'pressure': main['pressure'],      # Pression atmosphérique
        'humidity': main['humidity'],      # Humidité
        'weather': weather['description'], # Description du temps (ex : "clear sky")
        'local_time':  localtime.strftime('%Y-%m-%d %H:%M:%S')  # Formater la date et l'heure au format SQLite
    }
else:
    # En cas d'erreur dans la requête, stocker un message d'erreur
    current_weather_data = {"error": "Unable to fetch data"}

https://openweathermap.org/current

In [8]:
current_weather_data

{'city': 'Paris',
 'temperature': 22.72,
 'pressure': 1015,
 'humidity': 42,
 'weather': 'clear sky',
 'local_time': '2024-07-08 16:06:35'}

In [9]:
import sqlite3  # Importation du module sqlite3 pour travailler avec des bases de données SQLite

In [10]:
# Nom de la table
db_name = 'weather.db'

In [11]:
# Nom de la table
table_name = 'current'

In [12]:
if 'error' not in current_weather_data:
    # Établir une connexion à la base de données SQLite nommée 'weather.db'
    # Si cette base de données n'existe pas, elle sera créée dans le répertoire courant
    conn = sqlite3.connect(db_name)
    
    # Création d'un curseur pour permettre l'exécution de commandes SQL
    cursor = conn.cursor()
    
    # Exécution d'une commande SQL pour créer une table 'current' si elle n'existe pas déjà
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {table_name} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,  
            city TEXT,                             
            temperature REAL,                      
            pressure INTEGER,                      
            humidity INTEGER,                      
            description TEXT,
            local_time TEXT,
            UNIQUE(city, local_time)                 
        )
    ''')

    # Valider la transaction en cours et appliquer les modifications
    conn.commit()
    
    # Fermer la connexion à la base de données pour libérer les ressources
    conn.close()
    
    # Affichage d'un message 
    print(f"La table {table_name} a été créée avec succès.")
else:
    print(current_weather_data['error'])

La table current a été créée avec succès.


In [13]:
# Connexion à la base de données SQLite nommée 'weather_data.db'
# Si cette base de données n'existe pas, elle sera créée
conn = sqlite3.connect(db_name)

# Création d'un curseur permettant d'exécuter des commandes SQL
cursor = conn.cursor()

# Exécution d'une commande SQL pour insérer des données dans la table 'weather'
# Les valeurs à insérer sont spécifiées sous forme de paramètres pour éviter les injections SQL
cursor.execute(f'''
    INSERT INTO {table_name} (city, temperature, pressure, humidity, description, local_time)
    VALUES (?, ?, ?, ?, ?, ?)
''', (current_weather_data['city'], current_weather_data['temperature'], current_weather_data['pressure'], current_weather_data['humidity'], current_weather_data['weather'], current_weather_data['local_time']))

# Validation des modifications apportées à la base de données
conn.commit()

# Fermeture de la connexion à la base de données
conn.close()

print(f"Données insérées avec succès dans la table {table_name} de la base de données SQLite {db_name}.")

Données insérées avec succès dans la table current de la base de données SQLite weather.db.


In [14]:
# Connexion à la base de données SQLite
# La variable `db_name` doit contenir le nom de la base de données SQLite (par exemple, 'weather_data.db')
conn = sqlite3.connect(db_name)

In [15]:
# Création d'un curseur pour permettre l'exécution de commandes SQL
cursor = conn.cursor()

In [17]:
# Exécution d'une requête SQL pour récupérer toutes les lignes de la table `forecast`
# La variable `table_name` doit contenir le nom de la table à interroger (par exemple, 'weather')
cursor.execute(f"SELECT * FROM {table_name}")

# Récupérer toutes les lignes de résultats de la requête exécutée
rows = cursor.fetchall()

In [18]:
# Afficher les résultats récupérés
for row in rows:
    print(row)

# Fermer la connexion à la base de données pour libérer les ressources
conn.close()

(1, 'Paris', 22.72, 1015, 42, 'clear sky', '2024-07-08 16:06:35')
